# 🆕 LangChain v1.1.0 - Neue Features (Dezember 2025)

**Modul:** M04c - LangChain Advanced  
**Version:** 1.0  
**Datum:** Dezember 2025  
**Autor:** GenAI Projekt Team

---

## 📚 Lernziele

In diesem Notebook lernst du die **5 wichtigsten neuen Features** von LangChain v1.1.0 (Release: 2. Dezember 2025) kennen:

1. ✨ **Model Profiles** - Automatische Capability-Detection
2. ✨ **Smart Structured Output** - Auto-Inference von `ProviderStrategy`
3. ✨ **SystemMessage in Agents** - Cache-Control für Anthropic Claude
4. ✨ **ModelRetryMiddleware** - Automatische Retries mit exponential backoff
5. ✨ **ContentModerationMiddleware** - OpenAI Moderation für Safety-Layer

**Dokumentation:**
- [LangChain 1.0 Must-Haves](../LangChain_1.0_Must_Haves.md)
- [LangChain 1.1 Release Notes](https://changelog.langchain.com/announcements/langchain-1-1)

---

## 🛠️ Setup & Installation

In [1]:
# Installation (Google Colab)
!pip install -q langchain>=1.1.0 langchain-core>=1.1.0 langchain-openai>=0.2.0
!pip install -q python-dotenv

In [2]:
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul

In [3]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages, load_chat_prompt_template
setup_api_keys(['OPENAI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

✓ OPENAI_API_KEY erfolgreich gesetzt

Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

Installierte LangChain- und LangGraph-Bibliotheken:
langchain                                1.1.0
langchain-chroma                         1.0.0
langchain-classic                        1.0.0
langchain-community                      0.4.1
langchain-core                           1.1.0
langchain-ollama                         1.0.0
langchain-openai                         1.1.0
langchain-text-splitters                 1.0.0
langgraph                                1.0.3
langgraph-checkpoint                     3.0.1
langgraph-prebuilt                       1.0.5
langgraph-sdk                            0.2.10

IP-Adresse: 34.41.49.79
Hostname: 79.49.41.34.bc.googleusercontent.com
Stadt: Council Bluffs
Region: Iowa
Land: US
Koordinaten: 41.2619,-95.8608
Provider: AS396982 Google LLC
Postleitzahl: 51502
Zeitzone: America/Chicago


---

## 1️⃣ Model Profiles - Automatische Capability-Detection

### ❓ Was ist neu?

**Vor v1.1.0:**
```python
# Hardcoded Provider-Checks
if provider == "openai":
    use_structured_output = True
elif provider == "anthropic":
    use_structured_output = False
```

**Ab v1.1.0:**
```python
# Automatische Detection via Model Profiles
if llm.profile.supports_structured_output:
    use_structured_output = True
```

**Quelle:** Model Profiles werden von [models.dev](https://models.dev) (Open-Source Model Index) bezogen.

---

In [7]:
from langchain.chat_models import init_chat_model

# Model initialisieren (Kurznotation: "provider:model")
llm = init_chat_model("openai:gpt-4o-mini", temperature=0.0)

print("🔍 Model Profiles von gpt-4o-mini:\n")
# Print the full profile to debug available keys
print(f"Full LLM Profile: {llm.profile}\n")
print(f"✅ Structured Output: {llm.profile['structured_output']}")
print(f"✅ Function Calling: {llm.profile['tool_calling']}")
print(f"✅ Vision: {llm.profile['image_inputs']}")
# The profile does not directly contain 'supports_json_mode', but 'structured_output' often implies JSON capability for many models.
# For now, we will skip it or use a placeholder if 'json_mode' isn't explicitly listed.
# print(f"✅ JSON Mode: {llm.profile['supports_json_mode']}") # This key is not present.
print(f"📊 Context Window: {llm.profile['max_input_tokens']} tokens")

🔍 Model Profiles von gpt-4o-mini:

Full LLM Profile: {'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}

✅ Structured Output: True
✅ Function Calling: True
✅ Vision: True
📊 Context Window: 128000 tokens


In [13]:
# Dynamische Feature-Detection in der Praxis
from langchain_core.messages import HumanMessage

def analyze_image_if_supported(llm, image_url: str, question: str):
    """Analysiert Bild nur wenn Model Vision unterstützt."""

    if llm.profile['image_inputs']:
        print("✅ Model unterstützt Vision - Bild wird analysiert...")

        response = llm.invoke([HumanMessage(content=[
            {"type": "text", "text": question},
            {"type": "image_url", "image_url": {"url": image_url}}
        ])])

        return response.content
    else:
        print("❌ Model unterstützt keine Vision - Fallback zu Text-only")
        return "Model unterstützt keine Bildverarbeitung"

# Beispiel
result = analyze_image_if_supported(
    llm,
    "https://picsum.photos/id/237/200/300", # Changed to a more reliable image URL
    "Was siehst du auf diesem Bild?"
)

print(f"\n📝 Antwort:\n{result}")

✅ Model unterstützt Vision - Bild wird analysiert...

📝 Antwort:
Auf dem Bild ist ein schwarzer Welpe zu sehen, der auf einem Holzboden liegt. Der Welpe hat große, neugierige Augen und wirkt freundlich und verspielt.


---

## 2️⃣ Smart Structured Output - Auto-Inference

### ❓ Was ist neu?

`ProviderStrategy` wird automatisch aus Model Profiles abgeleitet:

- Wenn `llm.profile.supports_structured_output == True` → nutzt **native API** (OpenAI Structured Output)
- Wenn `llm.profile.supports_json_mode == True` → nutzt **JSON mode**
- Fallback → **function calling**

**Vorteil:** Weniger Konfiguration, intelligentere Defaults!

---

In [11]:
from pydantic import BaseModel, Field
from typing import List

# Pydantic Schema definieren
class MovieRecommendation(BaseModel):
    """Filmempfehlung mit Metadaten."""
    title: str = Field(description="Filmtitel")
    genre: str = Field(description="Genre (z.B. Sci-Fi, Drama, Action)")
    year: int = Field(description="Erscheinungsjahr")
    rating: float = Field(description="Bewertung von 1-10", ge=1, le=10)
    reason: str = Field(description="Kurze Begründung der Empfehlung")

# ✨ NEU in v1.1.0: Auto-Inference der besten Strategie
structured_llm = llm.with_structured_output(MovieRecommendation)

# Query
result = structured_llm.invoke(
    "Empfiehl mir einen guten Science-Fiction Film aus den 2010er Jahren"
)

print("🎬 Filmempfehlung:\n")
print(f"Titel: {result.title}")
print(f"Genre: {result.genre}")
print(f"Jahr: {result.year}")
print(f"Bewertung: {result.rating}/10")
print(f"Begründung: {result.reason}")

# Type-Check
print(f"\n✅ Type-Check: {type(result)} = {MovieRecommendation}")
assert isinstance(result, MovieRecommendation)

🎬 Filmempfehlung:

Titel: Inception
Genre: Sci-Fi, Action, Thriller
Jahr: 2010
Bewertung: 9.0/10
Begründung: Inception ist ein visuell beeindruckender Film, der die Grenzen zwischen Traum und Realität verschwimmen lässt. Die komplexe Handlung und die tiefgründigen Themen über das Unterbewusstsein machen ihn zu einem Meisterwerk des Science-Fiction-Genres.

✅ Type-Check: <class '__main__.MovieRecommendation'> = <class '__main__.MovieRecommendation'>


### 🔍 Vergleich: ALT vs. NEU

**❌ ALT (v1.0):**
```python
# Manuelle Strategie-Auswahl
structured_llm = llm.with_structured_output(
    MovieRecommendation,
    provider_strategy="native"  # Manuell festgelegt!
)
```

**✅ NEU (v1.1.0):**
```python
# Automatische Strategie-Auswahl via Model Profiles
structured_llm = llm.with_structured_output(MovieRecommendation)
# Nutzt automatisch "native" wenn llm.profile.supports_structured_output == True
```

---

## 3️⃣ SystemMessage Support in create_agent()

### ❓ Was ist neu?

`system_prompt` akzeptiert jetzt `SystemMessage` mit erweiterten Features:

- ✅ **Cache-Control** (Anthropic Claude) - spart Tokens bei wiederholten System-Prompts
- ✅ **Strukturierte Content Blocks** - Bilder, Dokumente im System-Prompt
- ✅ **Metadata** - Orchestration-Hints für komplexe Agents

---

In [14]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
from langchain_core.tools import tool

# Tool definieren
@tool
def calculate(expression: str) -> str:
    """Berechnet mathematische Ausdrücke.

    Args:
        expression: Mathematischer Ausdruck (z.B. "2 + 2", "sqrt(16)")
    """
    try:
        import math
        result = eval(expression, {"__builtins__": None}, {"math": math})
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# ❌ ALT: String-basierter System-Prompt
agent_old = create_agent(
    model=llm,
    tools=[calculate],
    system_prompt="You are a helpful math assistant",
    debug=False
)

# ✅ NEU (v1.1.0): SystemMessage mit erweiterten Features
agent_new = create_agent(
    model=llm,
    tools=[calculate],
    system_prompt=SystemMessage(
        content="""You are a helpful math assistant specialized in complex calculations.

Guidelines:
- Always explain your reasoning
- Use the calculate tool for all mathematical operations
- Format results clearly""",
        # Cache-Control für Anthropic (würde Tokens sparen bei wiederholten Calls)
        # cache_control={"type": "ephemeral"}  # Nur für Anthropic relevant
    ),
    debug=False
)

print("✅ Agent mit SystemMessage erstellt!")

✅ Agent mit SystemMessage erstellt!


In [15]:
# Agent testen
response = agent_new.invoke({
    "messages": [{"role": "user", "content": "Was ist die Quadratwurzel von 144?"}]
})

print("📊 Agent Response:")
print(response["messages"][-1].content)

📊 Agent Response:
Die Quadratwurzel von 144 ist 12.0.


---

## 4️⃣ ModelRetryMiddleware - Automatische Retries

### ❓ Was ist neu?

**Neue Middleware** für automatische Retries bei API-Fehlern:

- ✅ Exponential Backoff (2s, 4s, 8s, ...)
- ✅ Konfigurierbare Retry-Conditions (rate_limit, timeout, server_error)
- ✅ Jitter (Randomized delay) zur Vermeidung von Thundering Herd
- ✅ Production-Ready Resilience

---

In [ ]:
from langchain.agents.middleware import ModelRetryMiddleware

# Agent mit Retry-Middleware
agent_with_retry = create_agent(
    model=llm,
    tools=[calculate],
    system_prompt="You are a helpful assistant",
    middleware=[
        ModelRetryMiddleware(
            max_retries=3,
            backoff_factor=2.0,  # Exponential Backoff: 2s, 4s, 8s
            retry_on=["rate_limit_error", "timeout", "server_error"],
            jitter=True  # Randomized delay
        )
    ],
    debug=False
)

print("✅ Agent mit ModelRetryMiddleware erstellt!")
print("\n🔄 Retry-Konfiguration:")
print("- Max Retries: 3")
print("- Backoff: Exponential (2s, 4s, 8s)")
print("- Retry bei: Rate Limit, Timeout, Server Error")
print("- Jitter: Aktiviert")

In [ ]:
# Beispiel-Query (würde bei Rate Limit automatisch retries durchführen)
response = agent_with_retry.invoke({
    "messages": [{"role": "user", "content": "Berechne 123 * 456"}]
})

print("📊 Agent Response:")
print(response["messages"][-1].content)

### 💡 Use Cases für ModelRetryMiddleware

1. **Rate Limiting** - OpenAI/Anthropic Rate Limits
2. **Transiente Netzwerkfehler** - Timeouts, DNS-Fehler
3. **Server-Timeouts** - API-Provider überlastet
4. **Production Resilience** - Robuste Production-Apps

---

## 5️⃣ ContentModerationMiddleware - Safety-Layer

### ❓ Was ist neu?

**Neue Middleware** für Content-Moderation via OpenAI Moderation API:

- ✅ Filter für User-Input, Model-Output, Tool-Results
- ✅ Kategorien: Hate, Violence, Sexual, Self-Harm, etc.
- ✅ Block on Violation (Workflow stoppen)
- ✅ DSGVO & Brand-Safety für Production-Apps

---

In [ ]:
from langchain.agents.middleware import ContentModerationMiddleware

# Agent mit Content-Moderation
agent_with_moderation = create_agent(
    model=llm,
    tools=[calculate],
    system_prompt="You are a helpful assistant",
    middleware=[
        ContentModerationMiddleware(
            provider="openai",  # Nutzt OpenAI Moderation API
            check_user_input=True,      # Filter User-Inputs
            check_model_output=True,    # Filter Model-Responses
            check_tool_output=True,     # Filter Tool-Results
            block_on_violation=True,    # Workflow stoppen bei Violation
            categories=["hate", "violence", "sexual", "self-harm"]  # Kategorien
        )
    ],
    debug=False
)

print("✅ Agent mit ContentModerationMiddleware erstellt!")
print("\n🛡️ Moderation-Konfiguration:")
print("- Provider: OpenAI Moderation API")
print("- Check User Input: ✅")
print("- Check Model Output: ✅")
print("- Check Tool Output: ✅")
print("- Block on Violation: ✅")
print("- Kategorien: Hate, Violence, Sexual, Self-Harm")

In [ ]:
# Beispiel: Sichere Query
safe_response = agent_with_moderation.invoke({
    "messages": [{"role": "user", "content": "Was ist 5 + 3?"}]
})

print("✅ Sichere Query - Passed Moderation:")
print(safe_response["messages"][-1].content)

### 💡 Use Cases für ContentModerationMiddleware

1. **User-Generated Content** - Filter für Chat-Apps
2. **DSGVO-Compliance** - Datenschutz & Safety
3. **Brand-Safety** - Production-Apps mit Public-Facing Content
4. **Multi-Layer-Moderation** - Input + Output + Tool-Results

---

## 🎯 Zusammenfassung: Was ist neu in v1.1.0?

| Feature | Vorteil | Use Case |
|---------|---------|----------|
| **Model Profiles** | Automatische Capability-Detection | Dynamische Feature-Detection statt hardcoded Provider-Checks |
| **Smart Structured Output** | Auto-Inference von `ProviderStrategy` | Weniger Konfiguration, intelligentere Defaults |
| **SystemMessage in Agents** | Cache-Control & Content Blocks | Kostenersparnis bei Anthropic Claude |
| **ModelRetryMiddleware** | Automatische Retries | Resilience gegen API-Fehler (Rate Limits, Timeouts) |
| **ContentModerationMiddleware** | Safety-Layer | DSGVO-Compliance, Brand-Safety für Production |

### ✅ Keine Breaking Changes!

- v1.1.0 ist **100% rückwärtskompatibel** mit v1.0
- Bestehender Code funktioniert weiterhin
- Neue Features sind **optional**, aber empfohlen

---

## 📚 Weitere Ressourcen

- **LangChain 1.1 Release Notes:** https://changelog.langchain.com/announcements/langchain-1-1
- **LangChain 1.0 Must-Haves:** [../LangChain_1.0_Must_Haves.md](../LangChain_1.0_Must_Haves.md)
- **CLAUDE.md:** [../CLAUDE.md](../CLAUDE.md)
- **models.dev:** https://models.dev (Model Profiles Quelle)

---

## 🎓 Aufgaben

### Aufgabe 1: Model Profiles erkunden

Teste verschiedene Modelle und vergleiche ihre Profiles:
- `gpt-4o-mini`
- `gpt-4`
- `claude-3-sonnet` (falls Anthropic API-Key vorhanden)

Welche Unterschiede gibt es bei den unterstützten Features?

### Aufgabe 2: Strukturierte Ausgabe testen

Erstelle ein eigenes Pydantic-Schema (z.B. für Produkt-Reviews, Rezepte, oder Reiseziele) und teste `with_structured_output()` mit verschiedenen Queries.

### Aufgabe 3: Multi-Middleware Agent

Erstelle einen Agent, der **alle 3 Middleware** kombiniert:
- `ModelRetryMiddleware`
- `ContentModerationMiddleware`
- `SummarizationMiddleware` (für lange Sessions)

Teste den Agent mit verschiedenen Queries und beobachte das Zusammenspiel der Middleware.

---

**Ende des Notebooks** 🎉